# Рекомендательные системы

Вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

## 1. Метрика сходства
<b>1.1.</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

<b>1.2.</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [ ]:
import numpy as np
from utils import euclidean_similarity, pearson_similarity
import scipy.sparse as scp
from tqdm import tqdm

In [ ]:
def change(X):
    return np.where(X == -1, 0, X)

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [ ]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.metric = metric
        self.n_recommendations = n_recommendations
        self.alpha = alpha

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        self.X = change(X)

    def find_closest_users(self, user_id: int, n_closest_users: int):
        scores_corr = []
        if self.metric == 'euclidean':
            for i in range(len(self.X)):
                scores_corr.append((1 - euclidean_similarity(self.X[user_id], self.X[i]), i))
            sort_scores_corr = (sorted(scores_corr))[1:n_closest_users+1]
        elif self.metric == 'pearson':
            for i in range(len(self.X)):
                scores_corr.append((pearson_similarity(self.X[user_id], self.X[i]), i))
            sort_scores_corr = (sorted(scores_corr, reverse=True))[1:n_closest_users+1]
        sort_scores_corr = [item for item in sort_scores_corr if item[0] > self.alpha]
        near_idx = list(map(lambda x: x[1], sort_scores_corr))
        self.near_idx = near_idx
        return self.near_idx

    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        rating = []
        for i in range(self.X.shape[1]):
            sum1, sum2 = 0, 0
            for idx in self.near_idx:
                if self.metric == 'euclidean': 
                    sum1 += euclidean_similarity(self.X[user_id], self.X[idx]) * (self.X[idx][i] - np.mean(self.X[idx]))
                    sum2 += abs(euclidean_similarity(self.X[user_id], self.X[idx]))
                elif self.metric == 'pearson':
                    sum1 += pearson_similarity(self.X[user_id], self.X[idx]) * (self.X[idx][i] - np.mean(self.X[idx]))
                    sum2 += abs(pearson_similarity(self.X[user_id], self.X[idx]))
            if self.X[user_id][i] == 0:
                rating.append(((np.mean(self.X[user_id][self.X[user_id] > 0]) + (sum1 / sum2)), i))
        sort_rating = sorted(rating, reverse=True)
        return sort_rating[:n_recommendations]

<b>2.2. (1 балл)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

**Пример**: Допустим пользователи покупают товары и ставят им оценки.

U - юзер, которому хотим порекомендовать товары, которые он не покупал

Vi - i-ый юзер, который покупал товары


| Пользователь | Товар 1 | Товар 2 | Товар 3 | Товар 4 | Товар 5 | Товар 6 |
| :----------: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: |
|      U       |   -1    |    6    |    7    |   -1    |    5    |   -1    |
|      V1      |   8     |    4    |    2    |    7    |    6    |   -1    |
|      V2      |   -1    |   10    |    9    |    9    |   10    |    8    |
|      V3      |   5     |    4    |    6    |   -1    |    9    |    7    |
|      V4      |   7     |    6    |    7    |    4    |    6    |    5    |

P.S. Оценка -1 значит, что пользователь не покупал товар

In [ ]:
X = np.array([[-1, 6, 7, -1, 5, -1], [8, 4, 2, 7, 6, -1], [-1, 10, 9, 9, 10, 8], [5, 4, 6, -1, 9, 7], [7, 6, 7, 4, 6, 5]])

Проверим работу класса на метрике 'euclidean'

In [ ]:
user_based = UserBasedRecommendation(metric='euclidean')
user_based.fit(X)
near_idx = user_based.find_closest_users(0, 2)
user_based.make_recommendation(0, 1)

[(6.506495678375381, 0)]

Сравним рекомендации сделанные на основе метрики 'euclidean' и 'pearson', зададим параметр alpha=0.5

In [ ]:
def recom(metric):
    user_based = UserBasedRecommendation(metric=metric, alpha=0.5)
    user_based.fit(X)
    near_idx = user_based.find_closest_users(0, 2)
    recom = user_based.make_recommendation(0, 1)
    return [near_idx, recom]

In [ ]:
recom_eucl = recom('euclidean')
recom_pear = recom('pearson')

In [ ]:
print(f'Пользователи похожие на U: V{recom_eucl[0][0]} и V{recom_eucl[0][1]}')
print(f'Рекомендуемый товар на основе Евклидовой метрики: {recom_eucl[1][0][1] + 1}')

Пользователи похожие на U: V4 и V3
Рекомендуемый товар на основе Евклидовой метрики: 1


In [ ]:
print(f'Пользователи похожие на U: V{recom_pear[0][0]} и V{recom_pear[0][1]}')
print(f'Рекомендуемый товар на основе корреляции Пирсона: {recom_pear[1][0][1] + 1}')

Пользователи похожие на U: V2 и V4
Рекомендуемый товар на основе корреляции Пирсона: 4


<b>Объяснение:</b> рекомендуемые товары для метрик 'euclidean' и 'pearson' различаются, потому что отличаются похожие пользователи, в метрике 'pearson' модель показала похожего соседа V2, вместо V3, потому что в этой метрике в числители используется средняя оценка, а так как у пользователя V2 в среднем она очень высокая, поэтому корреляция Пирсона высокая, в Евклидовой метрики среднее не учитывается, поэтому V2 не будет близким соседом

## 3. Оценка качества

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

## 4. Применение модели

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

(100004, 4)

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(1, 671, 671)

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(1, 163949, 9066)

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [ ]:
data['userId'] = data['userId'] - 1

In [ ]:
movie_to_idx = {movie_name : idx for idx, movie_name in enumerate(data.movieId.unique())}
idx_to_movie = {i: movie_name for movie_name, i in movie_to_idx.items()}

In [ ]:
data['movieId'] = data['movieId'].apply(lambda x: movie_to_idx[x])

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(0, 670, 671)

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(0, 9065, 9066)

Удалим для наиболее активных пользователей 5 оценок

In [ ]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

((99954, 4), (50, 4))

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [ ]:
def get_sparse(data):
    return scp.coo_matrix(
        (
            data['rating'],  
            (data['userId'], data['movieId'])
        ), 
        shape=(len(np.unique(data['userId'])), len(movie_to_idx))
    ).tocsr()

In [ ]:
train_data = get_sparse(data)
train_data = train_data.todense().A
train_data

array([[2.5, 3. , 3. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [ ]:
test_users = list(np.unique(test_data['userId']))

In [ ]:
def calculate_mapk(metric, alpha, n_closest_users):
    actual = []
    for user in test_users:
        actual.append(np.array(test_data[test_data['userId'] == user]['movieId']))

    predicted = []
    top100 = UserBasedRecommendation(metric=metric, alpha=alpha)
    top100.fit(train_data)
    for user in test_users:
        top100.find_closest_users(user, n_closest_users)
        top100_user = top100.make_recommendation(user, 100)
        top100_user = list(map(lambda x: x[1], top100_user))
        predicted.append(top100_user)

    return np.array(actual), np.array(predicted)

In [ ]:
actual, predicted = calculate_mapk('euclidean', 0.8, 10)

In [ ]:
actual_pear, predicted_pear = calculate_mapk('pearson', 0.3, 10)

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [ ]:
from utils import mapk, apk

In [ ]:
print(f'Euclidean MAP@5 = {mapk(actual, predicted, 5)}')
print(f'Euclidean MAP@10 = {mapk(actual, predicted, 10)}')
print(f'Euclidean MAP@100 = {mapk(actual, predicted, 100)}')

Euclidean MAP@5 = 0.09566666666666668
Euclidean MAP@10 = 0.10011111111111111
Euclidean MAP@100 = 0.10741269924105748


In [ ]:
print(f'Pearson MAP@5 = {mapk(actual_pear, predicted_pear, 5)}')
print(f'Pearson MAP@10 = {mapk(actual_pear, predicted_pear, 10)}')
print(f'Pearson MAP@100 = {mapk(actual_pear, predicted_pear, 100)}')

Pearson MAP@5 = 0.09666666666666666
Pearson MAP@10 = 0.09666666666666666
Pearson MAP@100 = 0.10783826712503183


Как можно улучшить работу модели?

<b>Ответ:</b> Попробовать перебрать гиперпараметры модели (перебор ниже) или применить другую метрику близости пользователей, возможно с другой метрикой качество будет выше

In [ ]:
# euclidean
max_eucl = 0
alpha_euclidean = [0.3, 0.5, 0.7, 0.9]
clocest_users_eucl = [5, 10, 20, 30]
k = [5, 10, 100]
for alpha in tqdm(alpha_euclidean):
    for i in k:
        for user in clocest_users_eucl:
            actual_eucl, predicted_eucl = calculate_mapk('euclidean', alpha, user)
            if mapk(actual_eucl, predicted_eucl, i) > max_eucl:
                max_eucl = mapk(actual_eucl, predicted_eucl, i)
                best_k = i
                best_alpha = alpha
                best_user = user

100%|██████████| 4/4 [1:14:11<00:00, 1112.98s/it]


In [ ]:
print(f'Euclidean. Лучший результат метрики MAP@{best_k} = {round(max_eucl, 3)}, Лучшие гиперпарметры: k = {best_k}, alpha = {best_alpha}, closest_users = {best_user} ')

Euclidean. Лучший результат метрики MAP@100 = 0.107, Лучшие гиперпарметры: k = 100, alpha = 0.3, closest_users = 10 


In [ ]:
# pearson
max_pear = 0
alpha_pearson = [0.1, 0.2, 0.3]
clocest_users_pear = [5, 10, 30]
k = [5, 10, 100]
for alpha in tqdm(alpha_pearson):
    for i in k:
        for user in clocest_users_pear:
            actual_pear, predicted_pear = calculate_mapk('pearson', alpha, user)
            if mapk(actual_pear, predicted_pear, i) > max_pear:
                    max_pear = mapk(actual_pear, predicted_pear, i)
                    best_k_pear = i
                    best_alpha_pear = alpha
                    best_user_pear = user

100%|██████████| 3/3 [2:13:20<00:00, 2666.76s/it]


In [ ]:
print(f'Pearson. Лучший результат метрики MAP@{best_k_pear} = {round(max_pear, 3)}, Лучшие гиперпарметры: k = {best_k_pear}, alpha = {best_alpha_pear}, closest_users = {best_user_pear} ')

Pearson. Лучший результат метрики MAP@100 = 0.108, Лучшие гиперпарметры: k = 100, alpha = 0.3, closest_users = 30 


**Вывод**: в целом user-based рекомендательная система достаточно неплохо находит похожих пользователей, но в конечном счете фильмы рекомендует не очень хорошо, метрика MAP показала плохое качество

### Источники:
1. Преобразование датасета в матрицу рейтингов - ноутбук с семинара

2. Метрики mapk и apk - https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py